# LAB 5:  BigQuery ML Model Linear Feature Engineering/Transform.

**Learning Objectives**

1. Create and evaluate linear model with BigQuery's ML.FEATURE_CROSS
1. Create and evaluate linear model with BigQuery's ML.FEATURE_CROSS and ML.BUCKETIZE
1. Create and evaluate linear model with ML.TRANSFORM and L2 regularization


## Introduction 
In this notebook, we will create multiple linear models to predict the weight of a baby before it is born, using increasing levels of feature engineering using BigQuery ML.

In this lab, we will create and evaluate a linear model using BigQuery's ML.FEATURE_CROSS, create and evaluate a linear model using BigQuery's ML.FEATURE_CROSS and ML.BUCKETIZE, and create and evaluate a linear model using BigQuery's ML.TRANSFORM and L2 regularization.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/5_bqml_linear_transform_babyweight.ipynb).

## Load necessary libraries

Check that the Google BigQuery library is installed and if not, install it. 

In [ ]:
!pip freeze | grep google-cloud-bigquery==1.6.1 || pip install google-cloud-bigquery==1.6.1

## Verify tables exist

Verify that you previously created the dataset and data tables. If not, go back to lab [2_prepare_babyweight](../solutions/2_prepare_babyweight.ipynb) to create them.


In [ ]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_train
LIMIT 0

In [ ]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_eval
LIMIT 0

## Lab Task #1: Model 1:  Apply the ML.FEATURE_CROSS clause to categorical features

BigQuery ML now has ML.FEATURE_CROSS, a pre-processing clause that performs a feature cross.  

* ML.FEATURE_CROSS generates a STRUCT feature with all combinations of crossed categorical features, except for 1-degree items (the original features) and self-crossing items.  

* Syntax:  ML.FEATURE_CROSS(STRUCT(features), degree)

* The feature parameter is a categorical features separated by comma to be crossed. The maximum number of input features is 10. An unnamed feature is not allowed in features. Duplicates are not allowed in features.

* Degree(optional): The highest degree of all combinations. Degree should be in the range of [1, 4]. Default to 2.

Output: The function outputs a STRUCT of all combinations except for 1-degree items (the original features) and self-crossing items, with field names as concatenation of original feature names and values as the concatenation of the column string values.

#### INSTRUCTOR NOTE ONLY:  
Why ML.Feature_Cross uses STRUCT - All elements in an array have the same size because all elements are of the same datatype whereas STRUCT can contain elements of dissimilar datatype.  Hence, all elements are of different size.  For example, is_male and plurality are dissimilar datatypes.

#### Create model with feature cross.

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL
    babyweight.model_1

OPTIONS (
    MODEL_TYPE="LINEAR_REG",
    INPUT_LABEL_COLS=["weight_pounds"],
    DATA_SPLIT_METHOD="NO_SPLIT") AS

SELECT
    # TODO: Add base features and label
    ML.FEATURE_CROSS(
        # TODO: Cross categorical features
    ) AS gender_plurality_cross
FROM
    babyweight.babyweight_data_train

#### Create two SQL statements to evaluate the model.

In [ ]:
%%bigquery
SELECT
    *
FROM
    ML.EVALUATE(MODEL babyweight.model_1,
    (
    SELECT
        # TODO: Add same features and label as training
    FROM
        babyweight.babyweight_data_eval
    ))

In [ ]:
%%bigquery
SELECT
    # TODO: Select just the calculated RMSE
FROM
    ML.EVALUATE(MODEL babyweight.model_1,
    (
    SELECT
        # TODO: Add same features and label as training
    FROM
        babyweight.babyweight_data_eval
    ))

## BQML's Pre-processing functions:

Here are some of the preprocessing functions in BigQuery ML:
* ML.FEATURE_CROSS(STRUCT(features))    does a feature cross of all the combinations
* ML.POLYNOMIAL_EXPAND(STRUCT(features), degree)    creates x, x^2, x^3, etc.
* ML.BUCKETIZE(f, split_points)   where split_points is an array 

## Lab Task #2: Model 2:  Apply the BUCKETIZE Function 


##### BUCKETIZE 
Bucketize is a pre-processing function that creates "buckets" (e.g bins) - e.g. it bucketizes a continuous numerical feature into a string feature with bucket names as the value.

* ML.BUCKETIZE(feature, split_points)

* feature: A numerical column.

* split_points: Array of numerical points to split the continuous values in feature into buckets. With n split points (s1, s2 … sn), there will be n+1 buckets generated. 

* Output: The function outputs a STRING for each row, which is the bucket name. bucket_name is in the format of bin_<bucket_number>, where bucket_number starts from 1.

#### Apply the BUCKETIZE function within FEATURE_CROSS.
* Hint:  Create a model_2.

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL
    babyweight.model_2

OPTIONS (
    MODEL_TYPE="LINEAR_REG",
    INPUT_LABEL_COLS=["weight_pounds"],
    DATA_SPLIT_METHOD="NO_SPLIT") AS

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ML.FEATURE_CROSS(
        STRUCT(
            is_male,
            ML.BUCKETIZE(
                # TODO: Bucketize mother_age
            ) AS bucketed_mothers_age,
            plurality,
            ML.BUCKETIZE(
                # TODO: Bucketize gestation_weeks
            ) AS bucketed_gestation_weeks
        )
    ) AS crossed
FROM
    babyweight.babyweight_data_train

#### Create three SQL statements to EVALUATE the model.

In [ ]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL babyweight.model_2)

In [ ]:
%%bigquery
SELECT
    *
FROM
    ML.EVALUATE(MODEL babyweight.model_2,
    (
    SELECT
        # TODO: Add same features and label as training
    FROM
        babyweight.babyweight_data_eval))

In [ ]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL babyweight.model_2,
    (
    SELECT
        # TODO: Add same features and label as training
    FROM
        babyweight.babyweight_data_eval))

## Lab Task #3: Model 3:  Apply the TRANSFORM clause and L2 Regularization

Before we perform our prediction, we should encapsulate the entire feature set in a TRANSFORM clause.  BigQuery ML now supports defining data transformations during model creation, which will be automatically applied during prediction and evaluation. This is done through the TRANSFORM clause in the existing CREATE MODEL statement. By using the TRANSFORM clause, user specified transforms during training will be automatically applied during model serving (prediction, evaluation, etc.) 

In our case, we are using the TRANSFORM clause to separate out the raw input data from the TRANSFORMED features.  The input columns of the TRANSFORM clause is the query_expr (AS SELECT part).  The output columns of TRANSFORM from select_list are used in training. These transformed columns are post-processed with standardization for numerics and one-hot encoding for categorical variables by default. 

The advantage of encapsulating features in the TRANSFORM clause is the client code doing the PREDICT doesn't change, e.g. our model improvement is transparent to client code. Note that the TRANSFORM clause MUST be placed after the CREATE statement.

##### [L2 Regularization](https://developers.google.com/machine-learning/glossary/#L2_regularization) 
Sometimes, the training RMSE is quite reasonable, but the evaluation RMSE illustrate more error. Given the severity of the delta between the EVALUATION RMSE and the TRAINING RMSE, it may be an indication of overfitting. When we do feature crosses, we run into the risk of overfitting.

Overfitting is a phenomenon that occurs when a machine learning or statistics model is tailored to a particular dataset and is unable to generalize to other datasets. This usually happens in complex models, like deep neural networks.  Regularization is a process of introducing additional information in order to prevent overfitting.

Therefore, we will apply L2 Regularization to the model 3.  As a reminder, a regression model that uses the L1 regularization technique is called Lasso Regression while a regression model that uses the L2 Regularization technique is called Ridge Regression.  The key difference between these two is the penalty term.  Lasso shrinks the less important feature’s coefficient to zero thus, removing some features altogether.  Ridge regression adds “squared magnitude” of coefficient as a penalty term to the loss function.

L1 regularization adds an L1 penalty equal to the absolute value of the magnitude of coefficients. In other words, it limits the size of the coefficients. L1 can yield sparse models (i.e. models with few coefficients); Some coefficients can become zero and eliminated. Lasso regression uses this method.

L2 regularization adds an L2 penalty equal to the square of the magnitude of coefficients. L2 will not yield sparse models and all coefficients are shrunk by the same factor (none are eliminated). Ridge regression and SVMs use this method.

The regularization terms are ‘constraints’ by which an optimization algorithm must ‘adhere to’ when minimizing the loss function, apart from having to minimize the error between the true y and the predicted ŷ.  This in turn reduces model complexity, making our model simpler. A simpler model can reduce the chances of overfitting.

#### Apply the TRANSFORM clause and L2 Regularization to the model_3 and run the query.

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL
babyweight.model_3

TRANSFORM(
    # TODO: Add base features and label as you would in select
    # TODO: Add transformed features as you would in select
)

OPTIONS (
    MODEL_TYPE="LINEAR_REG",
    INPUT_LABEL_COLS=["weight_pounds"],
    # TODO: Add L2 Regularization,
    DATA_SPLIT_METHOD="NO_SPLIT") AS

SELECT
    *
FROM
    babyweight.babyweight_data_train

#### Create three SQL statements to EVALUATE model_3.

In [ ]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL babyweight.model_3)

In [ ]:
%%bigquery
SELECT
    *
FROM
    ML.EVALUATE(MODEL babyweight.model_3,
    (
    SELECT
        *
    FROM
        babyweight.babyweight_data_eval
    ))

In [ ]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL babyweight.model_3,
    (
    SELECT
        *
    FROM
        babyweight.babyweight_data_eval
    ))

## Lab Summary: 
In this lab, we created and evaluated a linear model using BigQuery's ML.FEATURE_CROSS, created and evaluated a linear model using BigQuery's ML.FEATURE_CROSS and ML.BUCKETIZE, and created and evaluated a linear model using BigQuery's ML.TRANSFORM and L2 regularization.

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License